In [0]:
%sql
show catalogs

In [0]:
%sql
use catalog practice

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS TestSchema

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS TestSchema.TestVolume

In [0]:
df = spark.read.format('csv').option('delimiter','	').option('header',True).option('inferSchema',True).load('/Volumes/practice/testschema/testvolume/Price.csv')
display(df)

In [0]:
from pyspark.sql.functions import col,split,regexp_replace
from pyspark.sql.types import DecimalType
df2 = df.withColumn('item_price',regexp_replace(col('item_price'),'\$','').cast('double'))\
    .withColumn('NetSalesAmount',col('quantity')*col('item_price'))\
        .withColumn('choice_description',regexp_replace(col('choice_description'),'[\[\]]',''))\
            .replace('NULL','Unknown')\
                .where('NetSalesAmount>10')\
                    .orderBy('NetSalesAmount')
display(df2)

In [0]:
print(f'Num of orders with more than 10 dollars orderamount is {df2.count()}')

In [0]:
df2.write.format('delta').mode('overwrite').saveAsTable('practice.TestSchema.fact_orders')